In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime

In [ ]:
sb = pd.read_csv('https://data.stadt-zuerich.ch/dataset/politik_stimmbeteiligung-vor-urnengangen/download/stimmbeteiligung.csv', parse_dates=[0, 2])
sb['Aktualisierungs_Datum'] = sb['Aktualisierungs_Datum'].dt.date # nur das Datum behalten (ohne Zeit)
sb

In [ ]:
#latest = sb[sb.Abstimmungs_Datum == '2021-09-26'].reset_index(drop=True)
latest = sb[sb.Abstimmungs_Datum == sb.Abstimmungs_Datum.max()].reset_index(drop=True)
latest

In [ ]:
latest_point = latest[latest.Aktualisierungs_Datum == latest.Aktualisierungs_Datum.max()].to_dict('records')[0]
latest_point

In [ ]:
#plt.style.use('default')
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots()

latest.plot(kind='line', y='Stimmbeteiligung_Prozent', x="Aktualisierungs_Datum", label="Stimmbeteiligung", ax=ax)
ax.legend().set_visible(False)

# immer von 0-100% zeigen
ax.set_ylim(bottom=0, top=100)

# Chart soll vom «ersten Datum - 2 Tage» bis «Abstimmungssonntag + 3 Tage» gehen
ax.set_xlim(
    left=latest.iloc[0]['Aktualisierungs_Datum'] - pd.DateOffset(2),
    right=latest.iloc[0]['Abstimmungs_Datum'] + pd.DateOffset(3)
)

ax.set_ylabel('Stimmbeteiligung in %')
ax.set_xlabel('Aktualisierungsdatum')

# Major ticks alle 5 Tage, ab dem 5.
day_locator = mdates.DayLocator(bymonthday=range(5, 31, 5))
ax.xaxis.set_major_locator(day_locator)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')

# Minor ticks off
ax.xaxis.set_minor_locator(ticker.NullLocator())

# Abstimmungssonntag als Rote Linie mit Text zeigen
plt.axvline(latest.iloc[0]['Abstimmungs_Datum'], color="red")
ax.annotate(f"Abstimmungssonntag {latest.iloc[0]['Abstimmungs_Datum'].strftime('%d.%m.%Y')}",
            xy=(latest.iloc[0]['Abstimmungs_Datum'], 50),
            color='red',
            xycoords='data',
            xytext=(50,-50),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='-', color='red', connectionstyle="angle3,angleA=90,angleB=0"),
            fontsize=12)

# Annotation für neuesten Datenpunkt
ax.annotate(f"{latest_point['Aktualisierungs_Datum'].strftime('%d.%m.%Y')}: {latest_point['Stimmbeteiligung_Prozent']}%",
            xy=(latest_point['Aktualisierungs_Datum'], latest_point['Stimmbeteiligung_Prozent']),
            color='black',
            alpha=0.5,
            xycoords='data',
            xytext=(30,30),
            textcoords='offset points',
            arrowprops=dict(headwidth=8, width=0.7, color='black', alpha=0.5, connectionstyle="angle3,angleA=0,angleB=-90"),
            fontsize=12)

plt.show()